In [13]:
import pandas as pd
import numpy as np

import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Border, Side, Font

import warnings
warnings.filterwarnings('ignore')

import json

with open("../linki.json", "r") as f:
    db_link = json.load(f)["generatory_path"]

In [14]:
from sqlite3 import connect
cnn = connect(db_link)
tab = pd.read_sql("SELECT * FROM baza_STOLARNIA", cnn)
cnn.close()

In [15]:
komplety_stelazy = pd.read_excel("Z:/450. PLANISTA - ZAOPATRZENIE/LISTA_KOMPLETOW_STELAZY.xlsx")

def pars_bryla(x):
  if type(x) != str:
    y = str(x)
    y = y.replace(".0", "").replace(".", ",")

    return y
  return x

komplety_stelazy["BRYLA_GEN"] = komplety_stelazy.BRYLA_GEN.apply(pars_bryla).astype(str)
# komplety_stelazy

##PRZYGOTOWANIE ZAMÓWIENIA

In [25]:
kompletacje = pd.read_excel("Z:/450. PLANISTA - ZAOPATRZENIE/zam_stol/ZAM_HUDSON_3_24_19.xlsx", sheet_name="Arkusz2")
kompletacje.rename(columns={"Unnamed: 0": "OPIS", kompletacje.columns[1]: "ILOSC", kompletacje.columns[2]: "KOMPLETACJA", kompletacje.columns[3]: "PALETA"}, inplace=True)

ostatnia_zajeta_paleta = 7703

In [26]:
kompletacje

,OPIS,ILOSC,KOMPLETACJA,PALETA
0,HUDSON [3,5,1,1
1,"HUDSON [2,5]",10,1,2
2,HUDSON [3],15,1,3
3,"HUDSON [3,5]",15,1,4
4,"HUDSON [3,5]",15,1,5
5,HUDSON [4],15,1,6
6,HUDSON [LH,5,1,7
7,HUDSON LH],5,1,7
8,HUDSON P62x62,10,1,8
9,HUDSON P82x62,5,1,8


In [27]:
kompl = kompletacje[kompletacje.KOMPLETACJA == 1]
kompl["PALETA"] = kompl.PALETA + ostatnia_zajeta_paleta

class Paleta:
  def __init__(self, model, nr_palety) -> None:
    self.model = model
    self.nr_palety = nr_palety
    pal = kompl[(kompl[kompl.columns[3]] == nr_palety)].merge(komplety_stelazy[["OPIS", "BRYLA_GEN"]], how="left", on="OPIS")

    paleta = pal.merge(tab[(tab.MODEL == model) & (tab.BRYLA.isin(pal.BRYLA_GEN.unique()))], how="left", right_on="BRYLA", left_on="BRYLA_GEN")
    paleta["ILOSC_ELEMENTOW"] = paleta.ILOSC * paleta._ILOSC

    def czas_kompl(czas_maszyny, ozn, ilosc):
      if czas_maszyny == 0:
        return 0
      if "MAG" in ozn:
        return 0
      return ilosc * 0.113

    paleta["CZAS_KOMPLETACJI"] = paleta.apply(lambda x: czas_kompl(x._CZAS, x.OZNACZ, x.ILOSC_ELEMENTOW), axis=1)
    paleta[["OPIS","MATERIAL", "NUMER", "ILOSC", "_ILOSC", "ILOSC_ELEMENTOW", "OZNACZ", "WYMIAR", "CZAS_KOMPLETACJI"]].sort_values(by=["MATERIAL"])

    podsumowanie_paleta = paleta[["MATERIAL", "NUMER", "ILOSC_ELEMENTOW","CZAS_KOMPLETACJI"]].groupby(["MATERIAL", "NUMER"]).sum().reset_index().merge(paleta[["MATERIAL", "NUMER", "WYMIAR", "OZNACZ"]].drop_duplicates(), how="left", on=["MATERIAL", "NUMER"])


    self.elementy = paleta
    self.podsumowanie = podsumowanie_paleta
    self.czas_kompletacji = podsumowanie_paleta.CZAS_KOMPLETACJI.sum()

  def Drukuj_raport(self, nr_zlecenia):
    wb = openpyxl.Workbook()
    sheet = wb.active

    ftd1 = Font(size=20, bold=True)
    a1 = sheet["A1"]
    a1.value = f"ZLECENIE NR: {nr_zlecenia}"
    a1.font = ftd1
    sheet.merge_cells('A1:G1')

    sheet["A3"] = "SERIA"
    sheet["A4"] = "WÓZEK"
    sheet["A5"] = "NR SYMB"
    sheet["A6"] = "DATA WYDANA"
    sheet["A7"] = "DATA ZAKOŃCZENIA"
    sheet["C3"] = f"{self.model}"
    sheet["C4"] = f"Wózek nr {self.nr_palety}"
    sheet["C5"] = f"{self.model[:3]}"
    thin = Side(border_style="thin", color="000000")
    for row in sheet[f"A3:D7"]:
      for cell in row:
        cell.border = Border(bottom=thin)



    sheet["A48"] = f"{self.model}"
    sheet["A49"] = f"Zestawienie dla serii {self.model} w zleceniu nr : {nr_zlecenia}"
    a =  "  ".join([f"{x[1].ILOSC}x {x[1].OPIS.replace(self.model, '').strip()}" for x in kompl[kompl.PALETA == self.nr_palety][["OPIS", "ILOSC"]].iterrows()])
    sheet["A51"] = f"Wózek nr : {self.nr_palety} - {self.model[:3]}: " + a

    sheet["A53"] = "LP"
    sheet["B53"] = "MATERIAL"
    sheet["C53"] = "NUMER"
    sheet["D53"] = "ILOSC"
    sheet["E53"] = "WYMIAR"
    sheet["F53"] = "OZNACZ"
    sheet["G53"] = "CZAS_KOMPL"
    DF = self.podsumowanie[["MATERIAL", "NUMER", "ILOSC_ELEMENTOW", "WYMIAR", "OZNACZ", "CZAS_KOMPLETACJI"]]
    rows = dataframe_to_rows(DF, header=False)#.reindex(pd.Index([x for x in range(1,self.podsumowanie.shape[0]+1)])),

    for r_idx, row in enumerate(rows,1):
       for c_idx, value in enumerate(row,1):
             sheet.cell(row=r_idx+52, column=c_idx, value=value)

    lp = 1
    for i in range(DF.shape[0]):
      sheet.cell(row=54+i, column=1, value=lp)
      lp += 1

    thin = Side(border_style="thin", color="000000")
    for row in sheet[f"A53:g{DF.shape[0]+53}"]:
        for cell in row:
          cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

    sheet[f"D{DF.shape[0]+54}"] = DF.ILOSC_ELEMENTOW.sum()
    sheet[f"G{DF.shape[0]+54}"] = DF.CZAS_KOMPLETACJI.sum()

    ft1 = Font(size=100, bold=True)
    d120 = sheet["B121"]
    d121 = sheet["B122"]
    d120.value = f"{self.model}"
    d121.value = f"{self.nr_palety}"
    d120.font = ft1
    d121.font = ft1

    wb.save(f"{nr_zlecenia.replace('/', '_')} {self.nr_palety}.xlsx")

  def __repr__(self):
    return f"paleta nr: {self.nr_palety} {self.czas_kompletacji:.0f}min"


In [28]:
kompl

,OPIS,ILOSC,KOMPLETACJA,PALETA
0,HUDSON [3,5,1,7704
1,"HUDSON [2,5]",10,1,7705
2,HUDSON [3],15,1,7706
3,"HUDSON [3,5]",15,1,7707
4,"HUDSON [3,5]",15,1,7708
5,HUDSON [4],15,1,7709
6,HUDSON [LH,5,1,7710
7,HUDSON LH],5,1,7710
8,HUDSON P62x62,10,1,7711
9,HUDSON P82x62,5,1,7711


In [ ]:
# Paleta()

In [29]:
palety = [Paleta("HUDSON", p) for p in kompl.PALETA.drop_duplicates().to_list()]
palety

[paleta nr: 7704 27min,
 paleta nr: 7705 81min,
 paleta nr: 7706 122min,
 paleta nr: 7707 132min,
 paleta nr: 7708 132min,
 paleta nr: 7709 132min,
 paleta nr: 7710 62min,
 paleta nr: 7711 20min]

In [30]:
sum([x.czas_kompletacji for x in palety])

709.64

In [ ]:
# palety[-1].elementy

In [31]:
for p in palety:
  p.Drukuj_raport("HUD 3/1/24")